In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import T5EncoderModel, T5Tokenizer, AutoTokenizer, PreTrainedModel # Added this import
from datasets import load_dataset, Dataset
from peft import LoraConfig, get_peft_model, TaskType
from sklearn.metrics import mean_squared_error, accuracy_score
from scipy.stats import spearmanr
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm
import json
import gc
from google.colab import drive


# Monta Drive
drive.mount('/content/drive')


!unzip /content/drive/MyDrive/t5eval_stdev6.zip


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  /content/drive/MyDrive/t5eval_stdev6.zip
replace t5eval_contrastive/training_args.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace t5eval_contrastive/tokenizer.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace t5eval_contrastive/tokenizer_config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace t5eval_contrastive/spiece.model? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace t5eval_contrastive/special_tokens_map.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace t5eval_contrastive/pytorch_model.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [2]:


class T5EncoderForRegression(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.encoder = T5EncoderModel.from_pretrained(model_name)
        hidden_size = self.encoder.config.d_model

        # Regressor più profondo
        self.regressor = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.LayerNorm(hidden_size // 2),
            nn.GELU(),
            nn.Dropout(0.15),
            nn.Linear(hidden_size // 2, 64),
            nn.LayerNorm(64),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(64, 1)
        )

        # Inizializzazione
        for module in self.regressor.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    nn.init.zeros_(module.bias)

        self.loss_fn = nn.MSELoss()
        self.config = self.encoder.config
        self.config.problem_type = "regression"
        self.config.num_labels = 1
        self.forward_count = 0

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        labels=None,
        **kwargs
    ):

        self.forward_count += 1
        # Encoder forward
        outputs = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=True
        )

        # Mean pooling with safety checks
        last_hidden = outputs.last_hidden_state  # [batch, seq_len, hidden]
        # Debug: Check for NaN in encoder output
        if self.forward_count <= 3 or torch.isnan(last_hidden).any():
            print(f"\n🔍 Forward pass #{self.forward_count}")
            print(f"  Encoder output shape: {last_hidden.shape}")
            print(f"  Encoder output has NaN: {torch.isnan(last_hidden).any()}")
            print(f"  Encoder output min/max: {last_hidden.min().item():.4f}/{last_hidden.max().item():.4f}")

        mask = attention_mask.unsqueeze(-1).float()  # [batch, seq_len, 1]

        # Prevent division by zero
        mask_sum = mask.sum(dim=1)
        mask_sum = torch.clamp(mask_sum, min=1e-9)

        # Weighted average by attention mask
        pooled = (last_hidden * mask).sum(dim=1) / mask_sum

        # Debug pooled output
        if self.forward_count <= 3 or torch.isnan(pooled).any():
            print(f"  Pooled output shape: {pooled.shape}")
            print(f"  Pooled has NaN: {torch.isnan(pooled).any()}")
            print(f"  Pooled min/max: {pooled.min().item():.4f}/{pooled.max().item():.4f}")
            print(f"  Regressor weight stats: mean={self.regressor[1].weight.mean().item():.4f}, std={self.regressor[1].weight.std().item():.4f}")
        # Regression head
        logits = self.regressor(pooled).squeeze(-1)  # [batch]

        loss = None
        if labels is not None:
            # Ensure labels are float and handle NaN values
            labels = labels.float()
            valid_mask = ~torch.isnan(labels) & ~torch.isinf(labels)

            if valid_mask.sum() > 0:
                loss = F.mse_loss(logits[valid_mask], labels[valid_mask])
            else:
                # If no valid labels, create a dummy loss
                loss = torch.tensor(0.0, device=logits.device, requires_grad=True)
                print(f"  ⚠️ No valid samples in batch!")
        return {
            "loss": loss,
            "logits": logits
        }


In [ ]:
import torch
import json
from tqdm import tqdm

# ===========================
# 1. RICARICA MODELLO SALVATO
# ===========================
print("\n🔄 Ricaricamento modello salvato...")
model_name = "google/flan-t5-xl"  # Torniamo a base
# Ricostruisci l'architettura
model = T5EncoderForRegression(model_name)
tokenizer= AutoTokenizer.from_pretrained('t5eval_contrastive')
# Riapplica LoRA con gli stessi parametri del training
lora_config = LoraConfig(
    task_type=TaskType.FEATURE_EXTRACTION,
    r=32,
    lora_alpha=64,
    lora_dropout=0.2,
    target_modules=["q", "v", "k", "o"],
    bias="none",
    inference_mode=True,
    modules_to_save=["regressor"]
)
model.encoder = get_peft_model(model.encoder, lora_config)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Carica i pesi salvati
weights_path = "./t5eval_contrastive/pytorch_model.bin"
state_dict = torch.load(weights_path, map_location=device)
model.load_state_dict(state_dict)

model.to(device)
model.eval()
print("✅ Modello ricaricato con successo!")

# ===========================
# 2. FUNZIONE DI PREDIZIONE
# ===========================
def get_prediction_regression(precontext, sentence, ending, homonym, judged_meaning, example_sentence):
    """Genera predizione per un singolo esempio"""
    story = f"{precontext}\n{sentence}\n{ending}"

    prompt = f"""Rate how plausible the meaning is in the context.
Answer ONLY with a number between 1 and 5.
You may use decimals (e.g., 2.543, 4.032).

Story:
{story}

Target word: {homonym}
Sense: {judged_meaning}
Example: {example_sentence}

Answer:"""

    inputs = tokenizer(
        prompt,
        max_length=320,
        truncation=True,
        padding='max_length',
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        outputs = model(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"]
        )
        pred_score = outputs["logits"].item()

    # Clamp tra 1 e 5 provo tra 1.99 e 4.01 per giocare con la stdev e ottenere risultati migliori
    return max(1.99, min(4.01, pred_score))

# ===========================
# 3. GENERAZIONE PREDIZIONI
# ===========================
print("\n🔮 Generazione predizioni sul test set...")

# Carica test.json
test_path = 'dataset/test.json'
with open(test_path, "r") as f:
    test_data = json.load(f)

predictions = []

for sample_id, item in tqdm(test_data.items(), desc="Generating predictions"):
    # Gestisci example_sentence mancante
    example_s = item.get('example_sentence', '')
    if not example_s and 'gloss' in item:
        example_s = item['gloss']

    # Genera predizione
    pred = get_prediction_regression(
        precontext=item['precontext'],
        sentence=item['sentence'],
        ending=item['ending'],
        homonym=item['homonym'],
        judged_meaning=item['judged_meaning'],
        example_sentence=example_s
    )

    predictions.append({
        "id": str(sample_id),
        "prediction": pred
    })

# ===========================
# 4. SALVATAGGIO FILE
# ===========================
output_file = "predictions.jsonl"
with open(output_file, "w") as f:
    for entry in predictions:
        f.write(json.dumps(entry) + "\n")

print(f"\n✅ File {output_file} creato con successo con {len(predictions)} predizioni!")
print(f"📊 Range predizioni: [{min(p['prediction'] for p in predictions):.2f}, {max(p['prediction'] for p in predictions):.2f}]")

# Mostra prime 5 predizioni come verifica
print("\n🔍 Prime 5 predizioni:")
for p in predictions[:5]:
    print(f"  ID: {p['id']}, Prediction: {p['prediction']:.4f}")


🔄 Ricaricamento modello salvato...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Modello ricaricato con successo!

🔮 Generazione predizioni sul test set...


Generating predictions:   0%|          | 0/930 [00:00<?, ?it/s]

...


🔍 Forward pass #2
  Encoder output shape: torch.Size([1, 320, 2048])
  Encoder output has NaN: False
  Encoder output min/max: -0.8758/0.8318
  Pooled output shape: torch.Size([1, 2048])
  Pooled has NaN: False
  Pooled min/max: -0.2783/0.2963
  Regressor weight stats: mean=0.9962, std=0.0105


Generating predictions:   0%|          | 3/930 [00:01<07:58,  1.94it/s]


🔍 Forward pass #3
  Encoder output shape: torch.Size([1, 320, 2048])
  Encoder output has NaN: False
  Encoder output min/max: -0.7592/0.7900
  Pooled output shape: torch.Size([1, 2048])
  Pooled has NaN: False
  Pooled min/max: -0.2628/0.3158
  Regressor weight stats: mean=0.9962, std=0.0105


Generating predictions: 100%|██████████| 930/930 [04:22<00:00,  3.54it/s]


✅ File predictions.jsonl creato con successo con 930 predizioni!
📊 Range predizioni: [1.99, 4.01]

🔍 Prime 5 predizioni:
  ID: 0, Prediction: 4.0100
  ID: 1, Prediction: 2.4718
  ID: 2, Prediction: 4.0100
  ID: 3, Prediction: 2.7912
  ID: 4, Prediction: 4.0100


In [4]:
from google.colab import files
!zip -r submit_dev.zip ./predictions.jsonl
files.download('submit_dev.zip')

  adding: predictions.jsonl (deflated 79%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>